In [1]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, StronglyConvexFunction, ConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step
from PEPit.constraint import Constraint

import numpy as np
import cvxpy as cp

In [2]:
# verification mode: PEP
problem = PEPit.PEP()

rho = 1

L_smooth = 1; mu = 0.001
# f = problem.declare_function(StronglyConvexFunction, mu=mu)
# g = problem.declare_function(StronglyConvexFunction, mu=mu)
# f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
# g = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L_smooth)
f = problem.declare_function(ConvexFunction)
g = problem.declare_function(ConvexFunction)
func = f + g
x_star, y_star, func_star = func.stationary_point(return_gradient_and_function_value=True)
z0 = problem.set_initial_point()

x1, grad_f_1, f_1 = proximal_step(z0, f, 1/rho)
z1, grad_g_1, g_1 = proximal_step(x1, g, 1/rho)
y1 = rho * (x1 - z1)
problem.add_constraint(Constraint(func_star - f_1 - g_1, equality_or_inequality="inequality"))

x2, _, f_2 = proximal_step(z1 - (1/rho) * y1, f, 1/rho)
z2, _, g_2 = proximal_step(x2 + (1/rho) * y1, g, 1/rho)
y2 = y1 + rho * (x2 - z2)
problem.add_constraint(Constraint(func_star - f_2 - g_2, equality_or_inequality="inequality"))

E_1 = (1/rho) * (y1 - y_star)**2 + rho * (z1 - x_star)**2
E_2 = (1/rho) * (y2 - y_star)**2 + rho * (z2 - x_star)**2
Delta_1 = rho * (x2 - z2)**2 + rho * (z2 - z1)**2

problem.set_performance_metric(E_2 - (E_1 - Delta_1))

diff = problem.solve(verbose=1, solver=cp.MOSEK)
print(f"{diff=}")

(PEPit) Setting up the problem: size of the main PSD matrix: 7x7
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (2 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
		 function 2 : Adding 6 scalar constraint(s) ...
		 function 2 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: -1.4225415290169963e-10
diff=-1.4225415290169963e-10
